# 0. `BFGS` Description
<font color="steelblue" size="4">

1.Website
---------
1. `Code`: https://github.com/trsav/bfgs/blob/master/BFGS.py

2.Newton method
----------------
1. The `classic Newton method` approximates the function to be optimised $f(x)$ as a quadratic using the `Taylor series expansion`:
$\begin{equation}
f(x+p) = f(x) + \nabla f(x)^Tp + \frac{1}{2}p^T\nabla^2f(x)p \\
\end{equation}$
2. By `minimising this function` with respect to $p$, `the optimal search direction` $p$ can be found as:
$\begin{equation}
-\nabla^2f(x_{k})^{-1} \nabla f(x_k)
\end{equation}$
3. The `step length` $\alpha$ is then computed via a `backtrack linesearch` using `Wolfe conditions` that assure sufficient descrease.
4. The `inverse of the Hessian matrix` $\nabla^2 f^{-1}$ is `computationally expensive` to compute due to both finite difference limitations and the cost of inverting a particularly large matrix. 

3.BFGS
------
1. For this reason an approximation to the `inverse of the Hessian` is used $H$, This approximation is updated at each iteration based on:
    - `the change in` $x$ 
    - `the change in`  $\nabla f$
$\begin{equation}
H_{k+1} = (I-\frac{sy^T}{y^Ts})H_k(I-\frac{ys^T}{y^Ts}) + \frac{ss^T}{y^Ts}
\end{equation}$

</font>

# 1. Example 
<font color="steelblue" size="4">

1. Testing the `BFGS algorithm` on the `Rosenbrock function` in 2 dimensions, an optimal solution is found in `34 iterations`.
2. The code implements an `initial inverse of Hessian` $H_0$ as the `identity matrix`.
    - If the problem is two dimensional then the code can produce a trajectory plot of the optimisation scheme. 
    - The `central difference method` is used for the calculation of nablas.

</font>

In [273]:
import numpy as np
import matplotlib.pyplot as plt

## 1.1. Rosenbrock function
<font color="steelblue" size="4">

`Website`
---------
1. https://www.sfu.ca/~ssurjano/rosen.html

`Rosenbrock function formula`:
-----------------------------
$\begin{equation}
f(\vec{x}) = \sum_{i=1}^{d-1}\left[ 100(x_{i+1}-x_i^2)^2 + (x_i-1)^2 \right]
\end{equation}$

`Vectorize`:
------------
$\begin{aligned}
dia &=
\begin{bmatrix}
x_1 & 0 & \cdots & 0 \\
0 & x_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & x_N \\
\end{bmatrix}_{N*N}
\newline
offdia &=
\begin{bmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & 1 \\
0 & 0 & 0 & \cdots & 0 \\
\end{bmatrix}_{N*N}
\newline
-dia+offdia &= 
\begin{bmatrix}
-x_1 & 1 & 0 & \cdots & 0 & 0 \\
0 & -x_2 & 1 & \cdots & 0 & 0  \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & -x_{N-1} & 1 \\
0 & 0 & 0 & \cdots & 0 & -x_N\\
\end{bmatrix}_{N*N}
\newline
(-dia+offdia)\vec{x} &= 
\begin{bmatrix}
-x_1 & 1 & 0 & \cdots & 0 & 0 \\
0 & -x_2 & 1 & \cdots & 0 & 0  \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & -x_{N-1} & 1 \\
0 & 0 & 0 & \cdots & 0 & -x_N\\
\end{bmatrix}_{N*N}
\begin{bmatrix}
x_1 \\
x_2 \\
\vdots \\
x_{N-1} \\
x_N 
\end{bmatrix}=
\begin{bmatrix}
x_2 - x_1^2 \\
x_3 - x_2^2 \\
\vdots \\
x_N - x_{N-1}^2 \\
-x_N^2
\end{bmatrix}
\end{aligned}$

</font>

In [290]:
def rosenbrock_fun(x_array: np.array):
    '''
    Description
    -----------
        1. 计算 Rosenbrock 函数

    Parameters
    ----------
        1. x_array: np.array

    Return
    ------
        1. result: float
            一个数
    '''
    num_dims = x_array.shape[0]
    dia = np.diag(x_array)
    offdia = np.ones(num_dims - 1)
    offdia = np.diag(offdia, 1)
    """
    Input
    -----
        x_array: np.array([98, 99, 100])
    Output
    ------
        operator:
            [[ -98.    1.    0.]
            [   0.  -99.    1.]
            [   0.    0. -100.]]
    """
    #print(first_term)
    """
    一维 np.array 转换成 列向量形式(np.array):
        (n,) --> (1, n) --> (n, 1)
        arra_new = np.array([arr_old]).T
    """
    first_term = 100 * np.power((-dia + offdia) @ x_array, 2)   # first_term.shape = (n,)
    second_term = np.power(x_array - 1, 2)                      # second_term.shape = (n,)
    result = np.sum( (first_term+second_term)[:-1], axis=0 )    # result_term.shape = (n,)
    return result

In [291]:
x_array = np.array([-1.2, 1])

print(rosenbrock_fun(x_array=x_array))

24.199999999999996


## 1.2. Central finite difference calculation
<font color="steelblue" size="4">

`Vectorize`:
-----------
$\begin{aligned}
\vec{x} &= 
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_n
\end{bmatrix}
\newline
\vec{x}-h &= 
\begin{bmatrix}
x_1-h \\
x_2-h \\
x_3-h \\
\vdots \\
x_n-h
\end{bmatrix}
\newline
\vec{x}+h &= 
\begin{bmatrix}
x_1+h \\
x_2+h \\
x_3+h \\
\vdots \\
x_n+h
\end{bmatrix}
\newline
\vec{nabla}&=\frac{f(x_i+h)-f(x_i-h)}{2h}
\end{aligned}$

</font>

<font color="red" size="4">

Note
----
1. 求`偏导数`$\nabla f$，每次只能改变一个维度的值，所以只能用循环

</font>

In [297]:
def nabla(func, x_array):
    '''
    Description
    -----------
        1. CENTRAL FINITE DIFFERENCE CALCULATION

    Parameters
    ----------
        1. func:
            本例中使用 Rosenbrock 函数
        2. x_array: np.array
            一维形式, (num,)
    
    Return
    ------
        1. nabla: np.array
            一维形式, (num,)
    '''
    h = np.cbrt( np.finfo(float).eps )
    num_dims = x_array.shape[0]
    nabla_array = np.zeros(num_dims)

    for idx in range(num_dims):
        x_before = np.copy(x_array)
        x_next = np.copy(x_array)
        
        # 求偏导数，每次只能改变一个维度的值
        x_before[idx] -= h
        x_next[idx] += h

        nabla_array[idx] = (func(x_next) - func(x_before)) / (2*h)

    return nabla_array

In [298]:
x_array = np.array([-1.2, 1])

print( nabla(rosenbrock_fun, x_array) )

[-215.60000001  -88.        ]


## 1.3. Line Search: `BACKTRACK LINE SEARCH` WITH `WOLFE CONDITIONS`
<font color="steelblue" size="4">


1.Line Search
-------------
$\begin{equation}
z_{k+1} = z_k + \alpha_kp_k
\end{equation}$
- $p_k$: search `direction`
- $\alpha_k$: search `step length`



2.Wolfe Condition
-----------------
1. `Sufficient Decrease Condition (SDC)`
$\begin{equation}
f(x_k+\alpha_kp_k) \leq f(x_k) + c_1\alpha_k\nabla f_k^T p_k
\end{equation}$
```python
x_array_new = x_array + alpha * direction
condition_sdc = ( func(x_array_new) <= func(x_array) + (c1*alpha*nabla.T@direction_array) )
```
2. `Curvature Condition (CC)`
$\begin{equation}
\nabla f(x_k+\alpha_kp_k)^Tp_k \geq c_2 \nabla f_k^T p_k
\end{equation}$
```python
condition_cc = ( nabla_new.T@direction >= c2 * nabla.T@direction )
```
3. $c_1$, $c_2$ 满足
$\begin{equation}
0 < c_1 < c_2 < 1
\end{equation}$

In [299]:
def line_search(
            func,
            x_array,
            direction,
            nabla_array,
            ):
    '''
    Parameters
    ----------
        1. func: 
            优化的函数
        2. x_array:
            step k 的坐标
        3. direction: np.array
            下一步线搜索的方向
        4. nabla_array: np.array
            step k 的梯度
    Return
    ------
        1. alpha: float
            Search step length
    '''
    alpha = 1
    c1 = 1e-4
    c2 = 0.9
    x_array_new = x_array + alpha * direction
    nabla_array_new = nabla(func, x_array=x_array)
    

    """
    Note 
    ---- 
        1. Return of `(func(x_array_new) <= func(x_array) + (c1*alpha*nabla_array.T@direction))`
            [False False]
        2. Github 上写的是：
            - (func(x_array_new) >= func(x_array) + (c1*alpha*nabla_array.T@direction)).any()
            - (nabla_array_new.T@direction <= c2*nabla_array.T@direction).any()
    """
    while ( (func(x_array_new) >= func(x_array) + (c1*alpha*nabla_array.T@direction)).any() and \
            (nabla_array_new.T@direction <= c2*nabla_array.T@direction).any() ):
        # 当 alpha 不满足要求时，我们以 `alpha *= 0.5` 的方式更新 alpha
        alpha = alpha * 0.5
        x_array_new = x_array + alpha * direction
        nabla_new = nabla(func, x_array=x_array_new)

    return alpha

## 1.4. `BFGS` algorithm
<font color="steelblue" size="4">

1. For this reason an approximation to the `inverse of the Hessian` is used $H$, This approximation is updated at each iteration based on:
    - `the change in` $x$ 
    - `the change in`  $\nabla f$
$\begin{equation}
H_{k+1} = (I-\frac{sy^T}{y^Ts})H_k(I-\frac{ys^T}{y^Ts}) + \frac{ss^T}{y^Ts}
\end{equation}$

</font>

In [311]:
def BFGS(
        func,
        x0_array: np.array,
        max_num_iters: int,
        plot_mark: bool = False
        ):
    '''    
    Parameters
    ----------
        1. 

    Return
    ------
        1.
    '''
    ### Part I. Initialization
    num_dims = x0_array.shape[0]        # dimension of problem 
    nabla_array = nabla(func, x0_array)    # initial gradient 
    H = np.eye(num_dims)                # initial Hessian

    x_array = x0_array
    num_iters = 2


    ### Part II. Plot
    if plot_mark:
        if num_dims == 2:
            x_array_store = np.zeros((1,2))   # storing x values
            x_array_store[0, :] = x_array
        else:
            print("Too many dimensions to produce trajectory plot!")
            plot = False
    

    ### Part III. Loop to optimize
    while np.linalg.norm(nabla_array) > 1e-5:     # while gradient is positive
        if num_iters > max_num_iters:
            print("Maximum iterations reached!")
            break

        num_iters += 1
        direction = (-H@nabla_array)              # search direction (Newton Method), np.array
    
        alpha = line_search(func=func,      # seach step length, int
                            x_array=x_array,
                            direction=direction,
                            nabla_array=nabla_array)
        s = alpha * direction               # 1. calcualte `s` in BFGS equation 
        x_array_new = x_array + alpha * direction
        nabla_array_new = nabla(func=func, x_array=x_array_new)
        y = nabla_array_new - nabla_array               # 2. calculate `y` in BFGS equation
        
        ### Importance!!!
        # s.reshape(-1, 1), y.reshape(-1, 1): make s, y to be column vector
        s = np.array([s]).reshape(-1, 1)
        y = np.array([y]).reshape(-1, 1)

        H_left = np.eye(num_dims) - s@y.T / (y.T@s)[0, 0]
        H_right = np.eye(num_dims) - y@s.T / (y.T@s)[0, 0]
        addition_term = s@s.T / (y.T@s)[0, 0]
        H = H_left@H@H_right + addition_term    # 3. update Hessian matrix

        nabla_array = nabla_array_new
        x_array = x_array_new
        if plot_mark:
            x_array_store = np.append(x_array_store, [x_array], axis=0)     # storing x_array
    
    return x_array

In [312]:
x_opt = BFGS(rosenbrock_fun, np.array([-1.2, 1]), 100, plot_mark=False)
print(x_opt)

[0.99999999 0.99999998]
